In [36]:
#自分の学生証番号の下6桁に変更して実行してください
student_id =  200968

In [37]:
#この欄は変更せずに、実行してください

import random
i = student_id
n = 6
random.seed(i)
cost_ = []
value_ = []
for t in range(n):
    cost_.append(random.randint(1,8))
    value_.append(random.randint(2,16))
    print(t,'番目の要素x_', t, 'のコストは',cost_[-1],'、価値は',value_[-1])
budget_ = n*3

print('cost_ =', cost_)
print('value_ =', value_)
a = []
for i in range(n):
  a.append(value_[i]/cost_[i])

print('value/cost =',a)
print('budget_ =', budget_)

0 番目の要素x_ 0 のコストは 8 、価値は 15
1 番目の要素x_ 1 のコストは 3 、価値は 16
2 番目の要素x_ 2 のコストは 3 、価値は 9
3 番目の要素x_ 3 のコストは 5 、価値は 2
4 番目の要素x_ 4 のコストは 7 、価値は 7
5 番目の要素x_ 5 のコストは 2 、価値は 2
cost_ = [8, 3, 3, 5, 7, 2]
value_ = [15, 16, 9, 2, 7, 2]
value/cost = [1.875, 5.333333333333333, 3.0, 0.4, 1.0, 1.0]
budget_ = 18


# **課題**
---


各要素は、選ぶか選ばないかのどちらかしかとれないとします。
上のcostの合計がbudgetを超えない範囲で、valueの合計を最大化するような、要素の選び方choiceを求めてください。

## 問１
### 課題１を貪欲法で解くプログラムを、以下の欄に書いてください。
解答は、最適なchoiceをリストで表し、choiceのi番目の要素が
1ならば要素x_iを採用、0ならば不採用
となるようにして
(最適値, choice)　という形式で答えてください。
例）
(33, [1 ,0 ,0 ,1 ,0 ,1 ])

In [38]:
import numpy as np

# コストあたりの価値を求めてリストにする
value_per_cost = []

for i in range(0,n):
  value_per_cost.append(value_[i]/cost_[i]) #配列に一つずつ追加

# コストあたりの価値が高い順に、indexを取得
index = np.argsort(value_per_cost)[::-1]
'print(index)して, value_per_costと見比べよう'
print(index)


#欲張り法の本体
def greedy(cost, value, budget):
    x = []
    choice = [0]*n
    s = 0
    b_childa = budget

    for i in range(6):
    
      #コスパの良い順にx=1にして採用
      budget -= cost[index[i]]
      choice[index[i]] = 1
      s += value[index[i]]
      print(budget)
          
    
      #budgetを超える場合は採用をやめる
      if (budget < 0):
        budget + cost[index[i]]
        s -= value[index[i]]
        choice[index[i]] = 0
        break
    #価値を最大化する採用の仕方choiceと、そのときの価値の合計sを返す
    
    return (s,list(choice))
    

print(greedy(cost_, value_, budget_))

[1 2 0 5 4 3]
15
12
4
2
-5
(42, [1, 1, 1, 0, 0, 1])


## 問２
### 課題１を分枝限定法で解くプログラムを書くために必要な、以下の関数を定義してください。

In [54]:
#割り当てが途中までの段階で、costの合計がbudgetを超えてしまわないかを判定する
def is_feasible(end_of_branch, cost, budget):
  for i in range(len(end_of_branch)): 
    if end_of_branch[i] == 1:  
      budget -= cost[i] 

  if budget>=0:
    return True
  else: 
    return False

    
#採否の割り当てが済んだ段階で、割り当て済み分の価値の合計を計算する
def objective_value(end_of_branch, value):
  obj_val = 0 
  for i in range(len(end_of_branch)):
    if end_of_branch[i] == 1:
      obj_val += value[i]

  return obj_val 


#緩和問題を解く(cost,valueがコスパ順にソート済みの場合)
def linear_relaxed_value(end_of_branch, cost, value, index, budget):
    #割り当て済みの分だけ価値を計算し、その分のcostをbudgetから差し引く
    relaxed_value = 0 
    for i in range(len(end_of_branch)): 
      if end_of_branch[i] == 1:
        budget -= cost[i]
        relaxed_value += value[i]
    
    
    #まだ割り当てていない分を、コスパのよい順に採用していく
    i = len(end_of_branch)-1
    while i<n:
      if budget <= 0:
        break
      elif cost[i] <= budget:
        budget -= cost[i]
        relaxed_value += value[i] 
      else:
        percent = budget/cost[i]
        budget = 0
        relaxed_value += value[i]*percent
        
        
        #緩和問題で得られる価値の合計を返す
      return relaxed_value



## 問３
### 課題１を分枝限定法で解くプログラムを書いてください。必要があれば、問2で定義した関数を呼び出してください。

In [56]:
def branch_and_bound_for_01knapsack_problem(cost, value, budget):
    #コスパの高い順に、要素の属性（コスト、価値など）を呼び出せるように準備
    import copy
    value_per_cost = []
    for i in range(0,n):
      value_per_cost.append(value_[i]/cost_[i])
    # コストあたりの価値が高い順に、indexを取得
    index = np.argsort(value_per_cost)[::-1]
    
    cost_index = []
    value_index = []
    for i in index:
      cost_index.append(cost[i])
      value_index.append(value[i])
  

    stack = [[]] #未検討の分枝のリストであり、各分枝の先端における採否の割り当てのリストを要素とする
    best_value = 0
    best_solution = [0]*n
    
    #未検討の分枝がなくなるまで、未検討の分枝から一つ取り出して以下の操作をする
    stack.append([0])
    stack.append([1])
    print(stack)
    
    while len(stack)!=0:
        
        end_of_branch = stack.pop()
        print("stack:",stack)
        print("end_pf_branch",end_of_branch)

        #実行不能ならその分枝の検討を飛ばす
        if not is_feasible(end_of_branch,cost, budget):
            print(end_of_branch, 'is not feasible')
            continue
        #緩和問題を解いた結果、暫定解より良くなる見込みがなければ飛ばす
        if linear_relaxed_value(end_of_branch,cost,value,index,budget) <= best_value:
            print(end_of_branch, 'is bounded')
            continue
        #割り当てが済んだ分枝について価値を計算し、暫定解と比較して更新する
        if len(end_of_branch)==n:
          obj_val = objective_value(end_of_branch, value_index)
          if obj_val > best_value:
            best_value = obj_val
            best_solution = end_of_branch
        
          print('the best solution at the moment is ', end_of_branch)
        
        #分枝を生やす
        else:
          end_of_branch.append(0)
          new_branch_1 = end_of_branch
          stack.append(copy.deepcopy(new_branch_1))
          print(stack)
          end_of_branch.remove(0)
          end_of_branch.append(1)
          new_branch_2 = end_of_branch
          stack.append(copy.deepcopy(new_branch_2))
          print(stack,new_branch_1)
          
        
    return(best_value, best_solution)


print(branch_and_bound_for_01knapsack_problem(cost_, value_, budget_))

[[], [0], [1]]
stack: [[], [0]]
end_pf_branch [1]
[[], [0], [1, 0]]
[[], [0], [1, 0], [1, 1]] [1, 1]
stack: [[], [0], [1, 0]]
end_pf_branch [1, 1]
[[], [0], [1, 0], [1, 1, 0]]
[[], [0], [1, 0], [1, 1, 0], [1, 1, 1]] [1, 1, 1]
stack: [[], [0], [1, 0], [1, 1, 0]]
end_pf_branch [1, 1, 1]
[[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0]]
[[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0], [1, 1, 1, 1]] [1, 1, 1, 1]
stack: [[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0]]
end_pf_branch [1, 1, 1, 1]
[1, 1, 1, 1] is not feasible
stack: [[], [0], [1, 0], [1, 1, 0]]
end_pf_branch [1, 1, 1, 0]
[[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0, 0]]
[[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0, 0], [1, 1, 1, 0, 1]] [1, 1, 1, 0, 1]
stack: [[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0, 0]]
end_pf_branch [1, 1, 1, 0, 1]
[1, 1, 1, 0, 1] is not feasible
stack: [[], [0], [1, 0], [1, 1, 0]]
end_pf_branch [1, 1, 1, 0, 0]
[[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0, 0, 0]]
[[], [0], [1, 0], [1, 1, 0], [1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 1]] [1, 1, 1